In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd

# Ordner erstellen, falls nicht vorhanden
output_dir = "evaluation_output"
os.makedirs(output_dir, exist_ok=True)

# CSV-Datei laden
df = pd.read_csv("output/ant/summary_results.csv")

# Kategorien setzen
df['method'] = df['method'].astype('category')
df['sampling'] = df['sampling'].astype('category')
df['dimensionality'] = df['dimensionality'].astype('category')


# Ensure output directory exists
output_dir = "evaluation_output"
os.makedirs(output_dir, exist_ok=True)

# Main Effect: Method
plt.figure(figsize=(8, 5))
sns.barplot(x='method', y='max_fitness', data=df, errorbar='sd')
plt.title('Main Effect: Reduction Method')
plt.ylabel('Fitness Avg')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "main_effect_method.png"))
plt.close()
plt.show()

# Main Effect: Sampling
plt.figure(figsize=(6, 5))
sns.barplot(x='sampling', y='max_fitness', data=df, errorbar='sd')
plt.title('Main Effect: Data Sampling')
plt.ylabel('Fitness Avg')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "main_effect_sampling.png"))
plt.close()
plt.show()

# Main Effect: Dimensionality
plt.figure(figsize=(8, 5))
sns.barplot(x='dimensionality', y='max_fitness', data=df, errorbar='sd')
plt.title('Main Effect: Dimensionality')
plt.ylabel('Fitness Avg')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "main_effect_dimensionality.png"))
plt.close()
plt.show()

# Interaction: Method × Sampling
plt.figure(figsize=(10, 6))
sns.pointplot(x='method', y='max_fitness', hue='sampling', data=df, errorbar='sd', dodge=True)
plt.title('Interaction: Reduction Method × Data Sampling')
plt.ylabel('Fitness Avg')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "interaction_method_sampling.png"))
plt.close()
plt.show()

# Interaction: Method × Dimensionality
plt.figure(figsize=(10, 6))
sns.pointplot(x='dimensionality', y='max_fitness', hue='method', data=df, errorbar='sd', dodge=True)
plt.title('Interaction Effect: Method × Dimensionality')
plt.ylabel('Fitness Avg')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "interaction_method_dimensionality.png"))
plt.close()
plt.show()

# Interaction: Sampling × Dimensionality
plt.figure(figsize=(10, 6))
sns.pointplot(x='dimensionality', y='max_fitness', hue='sampling', data=df, errorbar='sd', dodge=True)
plt.title('Interaction Effect: Sampling × Dimensionality')
plt.ylabel('Fitness Avg')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "interaction_sampling_dimensionality.png"))
plt.close()
plt.show()